In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import category_encoders as ce   # version 1.2.8
import numpy as np
from sklearn.pipeline import Pipeline

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
import lightgbm as lgb
import tensorflow

/home/vignesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/vignesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/vignesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/vignesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

# Read data

In [4]:
data = pd.read_csv('../data/carInsurance_train.csv')

In [5]:
from sklearn.model_selection import train_test_split

# Holding out 20% of the sample for test dataset

* Performing stratified sampling
* X_train, y_train - training dataset
* X_test, y_test - test dataset (hold-out)

In [6]:
X = data.drop('CarInsurance', axis=1)
target = data['CarInsurance']
X_train, X_test, y_train, y_test = train_test_split(X, target,  test_size=0.2, random_state=1)

# CallStart and CallEnd is converted to call duration

In [7]:
call_duration_arr = []
for index, row in X_train.iterrows():
    
    call_start = row['CallStart'] 
    call_end = row['CallEnd']
    call_start_hr = call_start.split(':')[0]
    call_start_min = call_start.split(':')[1]
    call_start_sec = call_start.split(':')[2]
    
    call_end_hr = call_end.split(':')[0]
    call_end_min = call_end.split(':')[1]
    call_end_sec = call_end.split(':')[2]
    
    call_start_dt = dt.datetime(2013,12,30,int(call_start_hr),int(call_start_min),int(call_start_sec))
    call_end_dt = dt.datetime(2013,12,30,int(call_end_hr),int(call_end_min),int(call_end_sec))

    call_duration = (call_end_dt-call_start_dt).total_seconds()
    call_duration_arr.append(call_duration)
    
X_train['call_duration'] = call_duration_arr

call_duration_arr = []
for index, row in X_test.iterrows():
    
    call_start = row['CallStart'] 
    call_end = row['CallEnd']
    call_start_hr = call_start.split(':')[0]
    call_start_min = call_start.split(':')[1]
    call_start_sec = call_start.split(':')[2]
    
    call_end_hr = call_end.split(':')[0]
    call_end_min = call_end.split(':')[1]
    call_end_sec = call_end.split(':')[2]
    
    call_start_dt = dt.datetime(2013,12,30,int(call_start_hr),int(call_start_min),int(call_start_sec))
    call_end_dt = dt.datetime(2013,12,30,int(call_end_hr),int(call_end_min),int(call_end_sec))

    call_duration = (call_end_dt-call_start_dt).total_seconds()
    call_duration_arr.append(call_duration)
    
X_test['call_duration'] = call_duration_arr

/home/vignesh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/vignesh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Removing unneccessary columns - ID, CallStart, CallEnd, LastContactDay, Default

In [8]:

if len(set(['Id', 'CallStart', 'CallEnd', 'LastContactDay']) - set(list(X_train.columns)))<1:
    X_train = X_train.drop(['Id', 'CallStart', 'CallEnd', 'LastContactDay', 'Default'], axis=1)
    
if len(set(['Id', 'CallStart', 'CallEnd', 'LastContactDay']) - set(list(X_test.columns)))<1:
    X_test = X_test.drop(['Id', 'CallStart', 'CallEnd', 'LastContactDay', 'Default'], axis=1)

In [9]:
X_train.isnull().sum()

Age                    0
Job                   16
Marital                0
Education            130
Balance                0
HHInsurance            0
CarLoan                0
Communication        724
LastContactMonth       0
NoOfContacts           0
DaysPassed             0
PrevAttempts           0
Outcome             2434
call_duration          0
dtype: int64

In [10]:
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin


# Custom Transformer functions

* Imputation
* Log Transformation
* Normalization

In [11]:
class FillImputer(BaseEstimator, TransformerMixin):
    def __init__(self, fill_dict):
        super().__init__()
        self.fill_dict = fill_dict

    def fit(self, X, y=None):

        return self

    def transform(self, X, y=None):
#         X[:] = (X.to_numpy() - self.means_) / self.std_

        X_copy = X.copy()
        X_copy = X_copy.fillna(self.fill_dict)

        return X_copy
    
class kNNImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        super().__init__()
        self.kneigh = KNeighborsClassifier(n_neighbors=3)
        self.ce_one_hot = ce.OneHotEncoder(cols = ['Job','Marital'])
        self.ce_one_hot_model = None

    def fit(self, X, y=None):
#         X = X.to_numpy()
#         self.means_ = X.mean(axis=0, keepdims=True)
#         self.std_ = X.std(axis=0, keepdims=True)
        imputation_features_non_missing = X[~X.Education.isnull()][['Job', 'Marital', 'Age']]
        education_non_missing = X[~X.Education.isnull()]['Education']
        
        self.ce_one_hot_model = self.ce_one_hot.fit(imputation_features_non_missing)
        imputation_features_non_missing_encoded = self.ce_one_hot_model.transform(imputation_features_non_missing)
        
        self.kneigh.fit(X=imputation_features_non_missing_encoded, y=education_non_missing)
    
        return self
    

    def transform(self, X, y=None):
#         X[:] = (X.to_numpy() - self.means_) / self.std_
    
#         non_missing = X[~X.Education.isnull()]
#         missing = X[X.Education.isnull()]

        X_copy = X.copy()
    
        missing_ind = X_copy[X_copy.Education.isnull()].index
        education = X_copy.Education.copy()
       
        imputation_features_missing = X_copy[X_copy.Education.isnull()][['Job', 'Marital', 'Age']]
        education_missing = X_copy[X_copy.Education.isnull()]['Education']
                
        imputation_features_missing_encoded = self.ce_one_hot_model.transform(imputation_features_missing)
        missing_values_pred = self.kneigh.predict(imputation_features_missing_encoded)

        counter = 0
        for index in missing_ind:
            education[index] = missing_values_pred[counter]
            counter+=1
        
        X_copy['Education'] = education

        return X_copy

In [12]:
from sklearn.preprocessing import MinMaxScaler as min_max
from sklearn.preprocessing import StandardScaler

class LogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        super().__init__()
        self.columns = columns

    def fit(self, X, y=None):

        return self

    def transform(self, X, y=None):
        
        X_copy = X.copy()

        for col in self.columns:
            X_copy[col] = np.log(X_copy[col]+1)
        

        return X_copy

class MinMaxScalarMultiple(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        super().__init__()
        self.columns = columns
        self.normalizer = None

    def fit(self, X, y=None):
        self.normalizer = min_max()
        self.normalizer.fit(X[self.columns])

        return self

    def transform(self, X, y=None):
        
        X_copy = X.copy()

        normalized_vals = self.normalizer.transform(X[self.columns])
        
        for i in range(0, len(self.columns)):
            col = [row[i] for row in normalized_vals]
            X_copy[self.columns[i]] = col
        

        return X_copy
    
class StandardScalarMultiple(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        super().__init__()
        self.columns = columns
        self.normalizer = None

    def fit(self, X, y=None):
        self.normalizer = StandardScaler()
        self.normalizer.fit(X[self.columns])

        return self

    def transform(self, X, y=None):
        
        X_copy = X.copy()

        normalized_vals = self.normalizer.transform(X[self.columns])
        
        for i in range(0, len(self.columns)):
            col = [row[i] for row in normalized_vals]
            X_copy[self.columns[i]] = col
        

        return X_copy

In [13]:
def quarter_month(x):
    if x in ['jan', 'feb', 'mar', 'apr']:
        return '1'
    elif x in ['may', 'jun', 'jul', 'aug']:
        return '2'
    elif x in ['sep', 'oct', 'nov', 'dec']:
        return '3'
    
def job_categories(x):
    if x in ['blue-collar', 'entrepreneur', 'housemaid']:
        return 'job1'
    elif x in ['services', 'self-employed', 'admin.', 'technician', 'management', 'missing']:
        return 'job2'
    elif x in ['unemployed', 'retired', 'student']:
        return 'job3'
    else:
        return 'no_group'
    
def age_group(x):
    if x>=18 and x<=30:
        return '18-30'
    elif x>30 and x<=40:
        return '31-40'
    elif x>40 and x<=50:
        return '41-50'
    elif x>50 and x<=60:
        return '51-60'
    elif x>60:
        return '>60'
    
def duration_category(x):
    
    if (x>0) and (x<=10):
        return '0-10'
    elif (x>10) and (x<=20):
        return '10-20'
    elif (x>20) and (x<=30):
        return '20-30'
    elif (x>30):
        return '>30'
    
def no_of_contacts_category(x):
    
    
    if (x>0 and x<=3):
        return 'c1'
    elif (x>3 and x<=8):
        return 'c2'
    else:
        return 'c3'

# def duration_category(x):
    
#     if (x>0) and (x<=10):
#         return '0-10'
#     else: return '>10'
    

In [103]:
############### converted negative balance to zero
X_train['Balance'] = X_train.Balance.apply(lambda x : 0 if x<0 else x)

############### create a new variable for indicting whether a client is previously contacted - Boolean
X_train['IsPreviouslyContacted'] = X_train.DaysPassed.apply(lambda x : 0 if x==-1 else 1)

############### convert -ve values for days passed
X_train['DaysPassed'] = X_train.DaysPassed.apply(lambda x : 0 if x==-1 else x)

############### convert month to quadrimester
X_train['LastContactQuadrimester'] = X_train.LastContactMonth.apply(lambda x : quarter_month(x))

X_train['Job'] = X_train['Job'].fillna('missing')
X_train['JobCategory'] = X_train.Job.apply(lambda x : job_categories(x))
X_train['AgeCategory'] = X_train.Age.apply(lambda x : age_group(x))
X_train['CallDurationCategory'] = X_train.call_duration.apply(lambda x : duration_category(x/60))
X_train['NoOfContactsCategory'] = X_train.NoOfContacts.apply(lambda x : no_of_contacts_category(x))


In [104]:
X_train.NoOfContactsCategory.unique()

array(['c1', 'c2', 'c3'], dtype=object)

# Create a sklearn pipeline for feature engineering - imputation, encoding, scaling

In [325]:
log_transformer = LogTransformer(columns=['call_duration'])
standard_scaler = StandardScalarMultiple(columns=['Age', 'Balance', 'NoOfContacts', 'PrevAttempts', 'DaysPassed'])
fill_imputer = FillImputer(fill_dict={'Communication': 'missing', 'Job' : 'missing', 'Outcome' : 'not_contacted'})
knn_imputer = kNNImputer()
one_hot_encoder = ce.OneHotEncoder(cols=['NoOfContactsCategory','CallDurationCategory','JobCategory', 'AgeCategory', 'Marital', 'Education', 'Outcome', 'Communication', 'LastContactQuadrimester'])


pipe = Pipeline([('log_transformer', log_transformer), 
                 ('standard_scaler', standard_scaler),
                 ('fill_imputer', fill_imputer),
                 ('knn_imputer', knn_imputer),
                 ('one_hot_encoder', one_hot_encoder)])

pipe.fit(X_train)
X_train_features = pipe.transform(X_train)
X_train_features = X_train_features.drop(['NoOfContacts', 'LastContactMonth', 'Job', 'Age', 'call_duration', 'Balance'], axis=1)

In [326]:
X_train_features[0:10]

,Marital_1,Marital_2,Marital_3,Education_1,Education_2,Education_3,HHInsurance,CarLoan,Communication_1,Communication_2,Communication_3,DaysPassed,PrevAttempts,Outcome_1,Outcome_2,Outcome_3,Outcome_4,IsPreviouslyContacted,LastContactQuadrimester_1,LastContactQuadrimester_2,LastContactQuadrimester_3,JobCategory_1,JobCategory_2,JobCategory_3,AgeCategory_1,AgeCategory_2,AgeCategory_3,AgeCategory_4,AgeCategory_5,CallDurationCategory_1,CallDurationCategory_2,CallDurationCategory_3,CallDurationCategory_4,NoOfContactsCategory_1,NoOfContactsCategory_2,NoOfContactsCategory_3
2996,1,0,0,1,0,0,0,0,1,0,0,-0.464205,-0.373862,1,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0
3295,1,0,0,1,0,0,1,0,1,0,0,-0.464205,-0.373862,1,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0
2284,0,1,0,0,1,0,1,0,0,1,0,-0.464205,-0.373862,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0
2793,1,0,0,0,1,0,0,0,1,0,0,1.424987,2.276469,0,1,0,0,1,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0
787,1,0,0,0,0,1,0,0,1,0,0,-0.464205,-0.373862,1,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0
2046,1,0,0,0,0,1,0,0,1,0,0,-0.464205,-0.373862,1,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0
739,0,0,1,1,0,0,1,0,0,1,0,-0.464205,-0.373862,1,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0
2149,0,0,1,1,0,0,1,0,0,1,0,-0.464205,-0.373862,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0
3369,0,1,0,0,1,0,0,0,1,0,0,-0.464205,-0.373862,1,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1
3492,0,0,1,1,0,0,1,1,1,0,0,-0.464205,-0.373862,1,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0


# Create a logistic regression model and performing cross validation using training dataset

In [327]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [328]:
lg = LogisticRegression(max_iter=500)

In [331]:
from sklearn.model_selection import GridSearchCV
param_grid = {
   'C': [0.001, 0.01, 0.1, 1]
    }
search = GridSearchCV(lg, param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=8).fit(X_train_features, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  10 out of  12 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  12 out of  12 | elapsed:    0.1s finished


In [332]:
search.best_estimator_

LogisticRegression(C=0.1, max_iter=500)

# Create a random forest model and performing cross validation using training dataset

In [333]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [334]:
rf = RandomForestClassifier()

In [335]:
from sklearn.model_selection import GridSearchCV
param_grid = {
   'n_estimators': [5, 10, 20, 30, 50],
   'max_depth': [5, 10, 15, 20],
   'min_samples_split': [20, 30, 50, 70, 100]
    }
search = GridSearchCV(rf, param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=8).fit(X_train_features, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done 285 out of 300 | elapsed:    3.6s remaining:    0.2s
[Parallel(n_jobs=8)]: Done 300 out of 300 | elapsed:    3.8s finished


In [336]:
search.best_estimator_

RandomForestClassifier(max_depth=15, min_samples_split=20, n_estimators=20)

In [337]:
rf = RandomForestClassifier()

In [338]:
cross_val_score(rf, X_train_features, y_train, cv=3, scoring='precision')

array([0.77135678, 0.72009569, 0.73658537])

# Create a gradient boosting model and performing cross validation using training dataset

In [339]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

In [340]:
gd = GradientBoostingClassifier()

In [196]:
gd_precision_scores = cross_val_score(gd, X_train_features, y_train, cv=3, scoring='precision')

In [197]:
gd_precision_scores

array([0.79108635, 0.7840617 , 0.79569892])

In [382]:
from sklearn.model_selection import GridSearchCV
param_grid = {
   'n_estimators': [5, 10, 20, 30, 50],
   'max_depth': [5, 10, 15, 20],
   'min_samples_split': [20, 30, 50, 70, 100],
   'learning_rate': [0.001, 0.01, 0.1]
    }
search = GridSearchCV(gd, param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=8).fit(X_train_features, y_train)

Fitting 3 folds for each of 300 candidates, totalling 900 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  52 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 288 tasks      | elapsed:   11.3s
[Parallel(n_jobs=8)]: Done 538 tasks      | elapsed:   22.0s
[Parallel(n_jobs=8)]: Done 900 out of 900 | elapsed:   39.4s finished


In [383]:
search.best_estimator_

GradientBoostingClassifier(max_depth=5, min_samples_split=50, n_estimators=50)

In [384]:
search.best_params_

{'learning_rate': 0.1,
 'max_depth': 5,
 'min_samples_split': 50,
 'n_estimators': 50}

# Apply pipeline on test dataset

In [343]:
# X_test['Balance'] = X_test.Balance.apply(lambda x : 0 if x<0 else x)
# X_test['IsPreviouslyContacted'] = X_test.DaysPassed.apply(lambda x : 0 if x==-1 else 1)
# X_test['DaysPassed'] = X_test.DaysPassed.apply(lambda x : 0 if x==-1 else x)
# X_test['LastContactQuadrimester'] = X_test.LastContactMonth.apply(lambda x : quarter_month(x))
# X_test['Job'] = X_test['Job'].fillna('missing')
# X_test['JobCategory'] = X_test.Job.apply(lambda x : job_categories(x))
# X_test['AgeCategory'] = X_test.Age.apply(lambda x : age_group(x))
# X_train['CallDurationCategory'] = X_train.call_duration.apply(lambda x : duration_category(x/60))
# X_test_1 = log_transformer.transform(X_test)
# X_test_2 = standard_scaler.transform(X_test_1)
# X_test_3 = fill_imX_train['JobCategory'] = X_train.Job.apply(lambda x : job_categories(x))puter.transform(X_test_2)
# X_test_4 = knn_imputer.transform(X_test_3)
# X_test_5 = one_hot_encoder.transform(X_test_4)

############### converted negative balance to zero
X_test['Balance'] = X_test.Balance.apply(lambda x : 0 if x<0 else x)

############### create a new variable for indicting whether a client is previously contacted - Boolean
X_test['IsPreviouslyContacted'] = X_test.DaysPassed.apply(lambda x : 0 if x==-1 else 1)

############### convert -ve values for days passed
X_test['DaysPassed'] = X_test.DaysPassed.apply(lambda x : 0 if x==-1 else x)

############### convert month to quadrimester
X_test['LastContactQuadrimester'] = X_test.LastContactMonth.apply(lambda x : quarter_month(x))

X_test['Job'] = X_test['Job'].fillna('missing')
X_test['JobCategory'] = X_test.Job.apply(lambda x : job_categories(x))
X_test['AgeCategory'] = X_test.Age.apply(lambda x : age_group(x))
X_test['CallDurationCategory'] = X_test.call_duration.apply(lambda x : duration_category(x/60))
X_test['NoOfContactsCategory'] = X_test.NoOfContacts.apply(lambda x : no_of_contacts_category(x))


X_test_features = pipe.transform(X_test)
X_test_features = X_test_features.drop(['NoOfContacts', 'LastContactMonth', 'Job', 'Age', 'call_duration', 'Balance'], axis=1)

In [344]:
X_test_features[0:5]

,Marital_1,Marital_2,Marital_3,Education_1,Education_2,Education_3,HHInsurance,CarLoan,Communication_1,Communication_2,Communication_3,DaysPassed,PrevAttempts,Outcome_1,Outcome_2,Outcome_3,Outcome_4,IsPreviouslyContacted,LastContactQuadrimester_1,LastContactQuadrimester_2,LastContactQuadrimester_3,JobCategory_1,JobCategory_2,JobCategory_3,AgeCategory_1,AgeCategory_2,AgeCategory_3,AgeCategory_4,AgeCategory_5,CallDurationCategory_1,CallDurationCategory_2,CallDurationCategory_3,CallDurationCategory_4,NoOfContactsCategory_1,NoOfContactsCategory_2,NoOfContactsCategory_3
200,1,0,0,0,0,1,0,0,1,0,0,-0.464205,-0.373862,1,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0
1078,1,0,0,0,1,0,0,0,1,0,0,-0.464205,-0.373862,1,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0
610,1,0,0,1,0,0,0,1,1,0,0,-0.464205,-0.373862,1,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0
2159,0,1,0,0,0,1,1,0,1,0,0,-0.464205,-0.373862,1,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0
1169,1,0,0,1,0,0,0,0,1,0,0,-0.464205,-0.373862,1,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0


# Fit the logistic regression model on train dataset and predict on test dataset

In [345]:
lg = LogisticRegression(C=0.1, max_iter=500)
lg.fit(X_train_features, y_train)

LogisticRegression(C=0.1, max_iter=500)

In [346]:
lg_pred_train = lg.predict(X_train_features)

In [347]:
accuracy_score(y_train, lg_pred_train)

0.7909375

In [348]:
precision_score(y_train, lg_pred_train)

0.7980861244019138

In [349]:
recall_score(y_train, lg_pred_train)

0.6455108359133127

In [350]:
lg_pred = lg.predict(X_test_features)

In [351]:
accuracy_score(y_test, lg_pred)

0.7975

In [352]:
precision_score(y_test, lg_pred)

0.7798507462686567

In [353]:
recall_score(y_test, lg_pred)

0.6698717948717948

# Fit the random forest model on train dataset and predict on test dataset

In [354]:
rf = RandomForestClassifier(max_depth=15, min_samples_split=20, n_estimators=20)
rf.fit(X_train_features, y_train)

RandomForestClassifier(max_depth=15, min_samples_split=20, n_estimators=20)

In [355]:
rf.feature_importances_

array([0.01534925, 0.01146278, 0.00876435, 0.00783097, 0.00991637,
       0.01007904, 0.06872557, 0.01685202, 0.02711436, 0.03139232,
       0.00332624, 0.0529709 , 0.04958244, 0.02478031, 0.00660981,
       0.0861526 , 0.01317549, 0.        , 0.04788527, 0.01597685,
       0.01274597, 0.00729829, 0.01551576, 0.02055622, 0.01085288,
       0.00983135, 0.00648791, 0.01947525, 0.00912385, 0.17559961,
       0.13106333, 0.04211049, 0.00516573, 0.00879814, 0.00953914,
       0.00788914])

In [356]:
feature_imp_arr = pd.DataFrame({'columns': X_train_features.columns, 'feature_importances': rf.feature_importances_})

In [357]:
rf_pred_train = rf.predict(X_train_features)

In [358]:
accuracy_score(y_train, rf_pred_train)

0.834375

In [359]:
precision_score(y_train, rf_pred_train)

0.8250853242320819

In [360]:
recall_score(y_train, rf_pred_train)

0.7484520123839009

In [361]:
rf_pred = rf.predict(X_test_features)

In [362]:
accuracy_score(y_test, rf_pred)

0.8125

In [363]:
precision_score(y_test, rf_pred)

0.7773972602739726

In [364]:
recall_score(y_test, rf_pred)

0.7275641025641025

# Fit the gradient boosting model on train dataset and predict on test dataset

In [385]:
gb = GradientBoostingClassifier(max_depth=5, min_samples_split=50, n_estimators=50, learning_rate=0.1)
gb.fit(X_train_features, y_train)

GradientBoostingClassifier(max_depth=5, min_samples_split=50, n_estimators=50)

In [386]:
gb_train = rf.predict(X_train_features)

In [387]:
accuracy_score(y_train, gb_train)

0.834375

In [388]:
precision_score(y_train, gb_train)

0.8250853242320819

In [389]:
recall_score(y_train, gb_train)

0.7484520123839009

In [390]:
gb_pred = gb.predict(X_test_features)

In [391]:
accuracy_score(y_test, gb_pred)

0.815

In [392]:
precision_score(y_test, gb_pred)

0.7907801418439716

In [393]:
recall_score(y_test, gb_pred)

0.7147435897435898